# Probabilistic Algorithms

## Bayesian Optimization

In [1]:
def onemax(vector)
  return vector.inject(0){|sum, value| sum + value}
end

:onemax

In [2]:
def random_bitstring(size)
  return Array.new(size){ ((rand()<0.5) ? 1 : 0) }
end

:random_bitstring

In [3]:
def path_exists?(i, j, graph)
  visited, stack = [], [i]
  while !stack.empty?
    return true if stack.include?(j)
    k = stack.shift
    next if visited.include?(k)
    visited << k
    graph[k][:out].each {|m| stack.unshift(m) if !visited.include?(m)}    
  end
  return false
end

:path_exists?

In [4]:
def can_add_edge?(i, j, graph)
  return !graph[i][:out].include?(j) && !path_exists?(j, i, graph)
end

:can_add_edge?

In [5]:
def get_viable_parents(node, graph)
  viable = []
  graph.size.times do |i|
    if node!=i and can_add_edge?(node, i, graph)
      viable << i
    end
  end
  return viable
end

:get_viable_parents

In [6]:
def compute_count_for_edges(pop, indexes)
  counts = Array.new(2**(indexes.size)){0}
  pop.each do |p|
    index = 0
    indexes.reverse.each_with_index do |v,i|
      index += ((p[:bitstring][v] == 1) ? 1 : 0) * (2**i)
    end
    counts[index] += 1
  end
 return counts
end

:compute_count_for_edges

In [7]:
def fact(v)
  return v <= 1 ? 1 : v*fact(v-1)
end

:fact

In [8]:
def k2equation(node, candidates, pop)
  counts = compute_count_for_edges(pop, [node]+candidates)
  total = nil
  (counts.size/2).times do |i|
    a1, a2 = counts[i*2], counts[(i*2)+1]
    rs = (1.0/fact((a1+a2)+1).to_f) * fact(a1).to_f * fact(a2).to_f
    total = (total.nil? ? rs : total*rs)
  end
  return total
end

:k2equation

In [9]:
def compute_gains(node, graph, pop, max=2)
  viable = get_viable_parents(node[:num], graph)
  gains = Array.new(graph.size) {-1.0}
  gains.each_index do |i|
    if graph[i][:in].size < max and viable.include?(i)
      gains[i] = k2equation(node[:num], node[:in]+[i], pop)
    end
  end
  return gains
end

:compute_gains

In [10]:
def construct_network(pop, prob_size, max_edges=3*pop.size)
  graph = Array.new(prob_size) {|i| {:out=>[], :in=>[], :num=>i} }
  gains = Array.new(prob_size)  
  max_edges.times do
    max, from, to = -1, nil, nil
    graph.each_with_index do |node, i|
      gains[i] = compute_gains(node, graph, pop)
      gains[i].each_with_index {|v,j| from,to,max = i,j,v if v>max}
    end
    break if max <= 0.0
    graph[from][:out] << to
    graph[to][:in] << from
  end
  return graph
end

:construct_network

In [11]:
def topological_ordering(graph)
  graph.each {|n| n[:count] = n[:in].size}
  ordered,stack = [], graph.select {|n| n[:count]==0}  
  while ordered.size < graph.size
    current = stack.shift
    current[:out].each do |edge|
      node = graph.find {|n| n[:num]==edge}
      node[:count] -= 1
      stack << node if node[:count] <= 0
    end
    ordered << current
  end
  return ordered
end

:topological_ordering

In [12]:
def marginal_probability(i, pop)
  return pop.inject(0.0){|s,x| s + x[:bitstring][i]} / pop.size.to_f
end

:marginal_probability

In [13]:
def calculate_probability(node, bitstring, graph, pop)
  return marginal_probability(node[:num], pop) if node[:in].empty?
  counts = compute_count_for_edges(pop, [node[:num]]+node[:in])
  index = 0
  node[:in].reverse.each_with_index do |v,i|
    index += ((bitstring[v] == 1) ? 1 : 0) * (2**i)
  end  
  i1 = index + (1*2**(node[:in].size))
  i2 = index + (0*2**(node[:in].size)) 
  a1, a2 = counts[i1].to_f, counts[i2].to_f
  return a1/(a1+a2)
end

:calculate_probability

In [14]:
def probabilistic_logic_sample(graph, pop)
  bitstring = Array.new(graph.size)
  graph.each do |node|
    prob = calculate_probability(node, bitstring, graph, pop)
    bitstring[node[:num]] = ((rand() < prob) ? 1 : 0)
  end
  return {:bitstring=>bitstring}
end

:probabilistic_logic_sample

In [15]:
def sample_from_network(pop, graph, num_samples)
  ordered = topological_ordering(graph)  
  samples = Array.new(num_samples) do
    probabilistic_logic_sample(ordered, pop)
  end
  return samples
end

:sample_from_network

In [19]:
def search(num_bits, max_iter, pop_size, select_size, num_children)
  pop = Array.new(pop_size) { {:bitstring=>random_bitstring(num_bits)} }
  pop.each{|c| c[:cost] = onemax(c[:bitstring])}
  best = pop.sort!{|x,y| y[:cost] <=> x[:cost]}.first
  max_iter.times do |it|    
    selected = pop.first(select_size)
    network = construct_network(selected, num_bits)
    arcs = network.inject(0){|s,x| s+x[:out].size}
    children = sample_from_network(selected, network, num_children)
    children.each{|c| c[:cost] = onemax(c[:bitstring])}
    #children.each {|c| puts " >>sample, f=#{c[:cost]} #{c[:bitstring]}"}
    pop = pop[0...(pop_size-select_size)] + children
    pop.sort! {|x,y| y[:cost] <=> x[:cost]}
    best = pop.first if pop.first[:cost] >= best[:cost]
    #puts " >it=#{it}, arcs=#{arcs}, f=#{best[:cost]}, [#{best[:bitstring]}]"
    converged = pop.select {|x| x[:bitstring]!=pop.first[:bitstring]}.empty?
    break if converged or best[:cost]==num_bits
  end
  return best
end

:search

In [20]:
# problem configuration
num_bits = 20

# algorithm configuration
max_iter = 100
pop_size = 50
select_size = 15
num_children = 25

# execute the algorithm
best = search(num_bits, max_iter, pop_size, select_size, num_children)
puts "done! Solution:","f=#{best[:cost]}/#{num_bits}"," s=#{best[:bitstring]}"


done! Solution:
f=19/20
 s=[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Compact Genetic Algorithm

In [21]:
def onemax(vector)
  return vector.inject(0){|sum, value| sum + value}
end

:onemax

In [22]:
def generate_candidate(vector)
  candidate = {}
  candidate[:bitstring] = Array.new(vector.size)
  vector.each_with_index do |p, i|
    candidate[:bitstring][i] = (rand()<p) ? 1 : 0
  end
  candidate[:cost] = onemax(candidate[:bitstring])
  return candidate
end

:generate_candidate

In [23]:
def update_vector(vector, winner, loser, pop_size)
  vector.size.times do |i|  
    if winner[:bitstring][i] != loser[:bitstring][i]
      if winner[:bitstring][i] == 1
        vector[i] += 1.0/pop_size.to_f
      else 
        vector[i] -= 1.0/pop_size.to_f
      end
    end
  end
end

:update_vector

In [25]:
def search(num_bits, max_iterations, pop_size)
  vector = Array.new(num_bits){0.5}
  best = nil
  max_iterations.times do |iter|
    c1 = generate_candidate(vector)
    c2 = generate_candidate(vector)
    winner, loser = (c1[:cost] > c2[:cost] ? [c1,c2] : [c2,c1])
    best = winner if best.nil? or winner[:cost]>best[:cost]
    update_vector(vector, winner, loser, pop_size)
    #puts " >iteration=#{iter}, f=#{best[:cost]}, s=#{best[:bitstring]}"
    break if best[:cost] == num_bits
  end
  return best
end

:search

In [26]:
# problem configuration
num_bits = 32

# algorithm configuration
max_iterations = 200
pop_size = 20

# execute the algorithm
best = search(num_bits, max_iterations, pop_size)
puts "done! Solution:","f=#{best[:cost]}/#{num_bits}","s=#{best[:bitstring]}"


done! Solution:
f=32/32
s=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Cross-Entropy Method

In [27]:
def objective_function(vector)
  return vector.inject(0.0) {|sum, x| sum +  (x ** 2.0)}
end

:objective_function

In [28]:
def random_variable(minmax)
  min, max = minmax
  return min + ((max - min) * rand())
end

:random_variable

In [29]:
def random_gaussian(mean=0.0, stdev=1.0)
  u1 = u2 = w = 0
  begin
    u1 = 2 * rand() - 1
    u2 = 2 * rand() - 1
    w = u1 * u1 + u2 * u2
  end while w >= 1
  w = Math.sqrt((-2.0 * Math.log(w)) / w)
  return mean + (u2 * w) * stdev
end

:random_gaussian

In [30]:
def generate_sample(search_space, means, stdevs)
  vector = Array.new(search_space.size)
  search_space.size.times do |i|
    vector[i] = random_gaussian(means[i], stdevs[i])
    vector[i] = search_space[i][0] if vector[i] < search_space[i][0]
    vector[i] = search_space[i][1] if vector[i] > search_space[i][1]
  end
  return {:vector=>vector}
end

:generate_sample

In [31]:
def mean_attr(samples, i)
  sum = samples.inject(0.0) do |s,sample| 
    s + sample[:vector][i]
  end 
  return (sum / samples.size.to_f)
end

:mean_attr

In [32]:
def stdev_attr(samples, mean, i)
  sum = samples.inject(0.0) do |s,sample| 
    s + (sample[:vector][i] - mean)**2.0
  end 
  return Math.sqrt(sum / samples.size.to_f)
end

:stdev_attr

In [33]:
def update_distribution!(samples, alpha, means, stdevs)
  means.size.times do |i|
    means[i] = alpha*means[i] + ((1.0-alpha)*mean_attr(samples, i))
    stdevs[i] = alpha*stdevs[i]+((1.0-alpha)*stdev_attr(samples,means[i],i))
  end
end

:update_distribution!

In [34]:
def search(bounds, max_iter, num_samples, num_update, learning_rate)
  means = Array.new(bounds.size){|i| random_variable(bounds[i])}
  stdevs = Array.new(bounds.size){|i| bounds[i][1]-bounds[i][0]}
  best = nil
  max_iter.times do |iter|
    samples = Array.new(num_samples){generate_sample(bounds, means, stdevs)}
    samples.each {|samp| samp[:cost] = objective_function(samp[:vector])}
    samples.sort!{|x,y| x[:cost]<=>y[:cost]}
    best = samples.first if best.nil? or samples.first[:cost] < best[:cost]
    selected = samples.first(num_update)
    update_distribution!(selected, learning_rate, means, stdevs)
    #puts " > iteration=#{iter}, fitness=#{best[:cost]}"
  end
  return best
end

:search

In [35]:
# problem configuration
problem_size = 3
search_space = Array.new(problem_size) {|i| [-5, 5]}

# algorithm configuration
max_iter = 100
num_samples = 50
num_update = 5
l_rate = 0.7

# execute the algorithm
best = search(search_space, max_iter, num_samples, num_update, l_rate)
puts "done! Solution: ","f=#{best[:cost]}","s=#{best[:vector].inspect}"


done! Solution: 
f=2.0511013646169462e-18
s=[1.2934784391249768e-09, -4.819400717021263e-10, 3.81770427644035e-10]


## Population-Based Incremental Learning (PBIL)

In [36]:
def onemax(vector)
  return vector.inject(0){|sum, value| sum + value}
end

:onemax

In [37]:
def generate_candidate(vector)
  candidate = {}
  candidate[:bitstring] = Array.new(vector.size)
  vector.each_with_index do |p, i|
    candidate[:bitstring][i] = (rand()<p) ? 1 : 0
  end
  return candidate
end

:generate_candidate

In [38]:
def update_vector(vector, current, lrate)
  vector.each_with_index do |p, i|
    vector[i] = p*(1.0-lrate) + current[:bitstring][i]*lrate
  end
end

:update_vector

In [39]:
def mutate_vector(vector, current, coefficient, rate)
  vector.each_with_index do |p, i|
    if rand() < rate
      vector[i] = p*(1.0-coefficient) + rand()*coefficient
    end
  end
end

:mutate_vector

In [40]:
def search(num_bits, max_iter, num_samples, p_mutate, mut_factor, l_rate)
  vector = Array.new(num_bits){0.5}
  best = nil
  max_iter.times do |iter|
    current = nil
    num_samples.times do 
      candidate = generate_candidate(vector)
      candidate[:cost] = onemax(candidate[:bitstring])
      current = candidate if current.nil? or candidate[:cost]>current[:cost]
      best = candidate if best.nil? or candidate[:cost]>best[:cost]
    end
    update_vector(vector, current, l_rate)
    mutate_vector(vector, current, mut_factor, p_mutate)
    #puts " >iteration=#{iter}, f=#{best[:cost]}, s=#{best[:bitstring]}"
    break if best[:cost] == num_bits
  end
  return best
end

:search

In [41]:
# problem configuration
num_bits = 64

# algorithm configuration
max_iter = 100
num_samples = 100
p_mutate = 1.0/num_bits
mut_factor = 0.05
l_rate = 0.1

# execute the algorithm
best=search(num_bits, max_iter, num_samples, p_mutate, mut_factor, l_rate)
puts "done! Solution:","f=#{best[:cost]}/#{num_bits}","s=#{best[:bitstring]}"


done! Solution:
f=64/64
s=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Univariate Marginal Distribution

In [42]:
def onemax(vector)
  return vector.inject(0){|sum, value| sum + value}
end

:onemax

In [43]:
def random_bitstring(size)
  return Array.new(size){ ((rand()<0.5) ? 1 : 0) }
end

:random_bitstring

In [44]:
def binary_tournament(pop)
  i, j = rand(pop.size), rand(pop.size)
  j = rand(pop.size) while j==i
  return (pop[i][:fitness] > pop[j][:fitness]) ? pop[i] : pop[j]
end

:binary_tournament

In [45]:
def calculate_bit_probabilities(pop)
  vector = Array.new(pop.first[:bitstring].length, 0.0)
  pop.each do |member|
    member[:bitstring].each_with_index {|v, i| vector[i] += v}
  end
  vector.each_with_index {|f, i| vector[i] = (f.to_f/pop.size.to_f)}
  return vector
end

:calculate_bit_probabilities

In [46]:
def generate_candidate(vector)
  candidate = {}
  candidate[:bitstring] = Array.new(vector.size)
  vector.each_with_index do |p, i|
    candidate[:bitstring][i] = (rand()<p) ? 1 : 0
  end
  return candidate
end

:generate_candidate

In [47]:
def search(num_bits, max_iter, pop_size, select_size)
  pop = Array.new(pop_size) do
    {:bitstring=>random_bitstring(num_bits)}
  end
  pop.each{|c| c[:fitness] = onemax(c[:bitstring])}
  best = pop.sort{|x,y| y[:fitness] <=> x[:fitness]}.first
  max_iter.times do |iter|
    selected = Array.new(select_size) { binary_tournament(pop) }
    vector = calculate_bit_probabilities(selected)
    samples = Array.new(pop_size) { generate_candidate(vector) }
    samples.each{|c| c[:fitness] = onemax(c[:bitstring])}
    samples.sort!{|x,y| y[:fitness] <=> x[:fitness]}
    best = samples.first if samples.first[:fitness] > best[:fitness]
    pop = samples
    #puts " >iteration=#{iter}, f=#{best[:fitness]}, s=#{best[:bitstring]}"
  end
  return best
end

:search

In [48]:
# problem configuration
num_bits = 64

# algorithm configuration
max_iter = 100
pop_size = 50
select_size = 30

# execute the algorithm
best = search(num_bits, max_iter, pop_size, select_size)
puts "done! Solution:","f=#{best[:fitness]}","s=#{best[:bitstring]}"


done! Solution:
f=60
s=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
